In [1]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
import pandas as pd
import requests

In [ ]:
# Coloca tus credenciales aquí
CLIENT_ID = ''
CLIENT_SECRET = ''
# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
# Coloca tus credenciales de lastfm aquí
SHARED_SECRET = ''
API_KEY = ''

last_fm_connect = requests.get(f"http://www.last.fm/api/auth/?api_key={API_KEY}")

In [5]:
paises_europa_central = {
    "AT": "Austria",
    "DE": "Alemania",
    "CH": "Suiza",
    "LI": "Liechtenstein",
    "BE": "Bélgica",
    "LU": "Luxemburgo",
    "NL": "Países Bajos"}

In [6]:
def extraccion_por_paises(diccionario_zonas, años):
    todos_los_datos = [] # Esto es una lista de dicts donde se van guardando los diccionarios de cada país
    for pais in diccionario_zonas.keys():
        datos_rango = sp.search(q = f'year:{años}', limit=50, offset=0, type='track', market={pais}) # Cada una tiene que cambiar en q el rango de años
        datos_por_pais = {}
        datos_por_pais['pais'] = pais
        datos_por_pais['datos'] = datos_rango
        todos_los_datos.append(datos_por_pais)
    return todos_los_datos
   

In [7]:
def filtro_de_datos(lista_datos_zonas, datos):
    for datos_pais in lista_datos_zonas:
        for element in datos_pais['datos']['tracks']['items']:
            datos['pais'].append(datos_pais['pais'])
            datos['popularity'].append(element['popularity'])
            datos['nombre'].append(element['name'])
            artistas = []
            for artista in element['artists']:
                artistas.append(artista['name'])   
            artistas_str = ', '.join(artistas)
            datos['artista'].append(artistas_str)

In [8]:
p_central = extraccion_por_paises(paises_europa_central, '2015-2019')

In [9]:
dic_central = {'nombre': [], 'popularity': [], 'artista': [], 'pais': [], 'genero': []}

In [10]:
filtro_de_datos(p_central, dic_central)

In [11]:
dic_central

{'nombre': ['All The Stars (with SZA)',
  'Say You Love Me (Live at Ziggo Dome, Amsterdam)',
  'The Night We Met',
  'Liebe Grüße (mit Fatoni)',
  'Neymar',
  "I'm Better (feat. Lamb)",
  'Swim',
  'Matador (feat. Tommy)',
  'Softcore',
  'Holding Back the Years (Live at Ziggo Dome, Amsterdam)',
  'HIGHEST IN THE ROOM',
  'Enamorado',
  'Hauseingang',
  "Greenwood, Philip Selway, Edward O'Brien, Colin Greenwood: Pyramid Song (Arr. Lawson)",
  'Ohne mein Team',
  'So Beautiful (Live at Ziggo Dome, Amsterdam)',
  'HUMBLE.',
  "Durch die schweren Zeiten (I'll Carry You) [feat. Angus & Julia Stone] [MTV Unplugged 2] - Live vom Atlantik 2018",
  'Something Just Like This',
  'Your Mirror (Live at Ziggo Dome, Amsterdam)',
  'See You Again (feat. Kali Uchis)',
  'A New Flame (Live at Ziggo Dome, Amsterdam)',
  'Lush Life',
  'Pakete',
  'Falling Down - Bonus Track',
  'Слэмятся пацаны',
  "Where's My Love",
  'For Your Babies (Live at Ziggo Dome, Amsterdam)',
  'lovely (with Khalid)',
  'Stre

Llamada a API Last.FM y filtro para conseguir el género

In [12]:
p_central_last = requests.get(f"https://ws.audioscrobbler.com/2.0/?method=track.gettoptags&artist=Sasha&track=here+she+comes+again&api_key={API_KEY}&autocorrect=1&format=json")

In [13]:
p_central_last.json()['toptags']['tag'][0]['name']

KeyError: 'toptags'

In [14]:
lista_nombres_central = list(dic_central.values())[0]
lista_artistas_central = list(dic_central.values())[2]

In [15]:
def llamada_lastfm(lista_nombres, lista_artistas,datos):
    for i in range(len(lista_nombres)):
        try:
            nombre = str(lista_nombres[i]).replace(' ', '+')
            artista = lista_artistas[i].replace(' ', '+').replace(',', '+&')
            llamada_genero = requests.get(f"https://ws.audioscrobbler.com/2.0/?method=track.gettoptags&artist={artista}&track={nombre}&api_key={API_KEY}&autocorrect=1&format=json")
            try:
                genero_cancion = llamada_genero.json()['toptags']['tag'][0]['name']
                datos['genero'].append(genero_cancion)
            except KeyError:
                datos['genero'].append("NULL")
                print(f"La canción {lista_nombres[i]} no está registrada en lastfm")
                
            except IndexError:
                datos['genero'].append("NULL")
                print(f"La canción {lista_nombres[i]} no tiene tags")
            
        except:
            datos['genero'].append("NULL")
            print(f"Hay problemas con la cación {lista_nombres[i]}")

In [16]:
llamada_lastfm(lista_nombres_central, lista_artistas_central, dic_central)

La canción All The Stars (with SZA) no está registrada en lastfm
La canción Say You Love Me (Live at Ziggo Dome, Amsterdam) no está registrada en lastfm
La canción The Night We Met no está registrada en lastfm
La canción Liebe Grüße (mit Fatoni) no está registrada en lastfm
La canción Neymar no está registrada en lastfm
La canción I'm Better (feat. Lamb) no está registrada en lastfm
La canción Swim no está registrada en lastfm
La canción Matador (feat. Tommy) no está registrada en lastfm
La canción Softcore no está registrada en lastfm
La canción Holding Back the Years (Live at Ziggo Dome, Amsterdam) no está registrada en lastfm
La canción HIGHEST IN THE ROOM no está registrada en lastfm
La canción Enamorado no está registrada en lastfm
La canción Hauseingang no está registrada en lastfm
La canción Greenwood, Philip Selway, Edward O'Brien, Colin Greenwood: Pyramid Song (Arr. Lawson) no está registrada en lastfm
La canción Ohne mein Team no está registrada en lastfm
La canción So Beauti

In [17]:
df_central = pd.DataFrame(dic_central)

In [18]:
df_central.index.name="id"

In [19]:
df_central

,nombre,popularity,artista,pais,genero
id,,,,,
0,All The Stars (with SZA),90,"Kendrick Lamar, SZA",AT,NULL
1,"Say You Love Me (Live at Ziggo Dome, Amsterdam)",1,Simply Red,AT,NULL
2,The Night We Met,84,Lord Huron,AT,NULL
3,Liebe Grüße (mit Fatoni),0,"Antilopen Gang, Fatoni",AT,NULL
4,Neymar,63,"Capital Bra, Ufo361",AT,NULL
...,...,...,...,...,...
345,Skrrt,7,"K27, Einar",NL,NULL
346,Lovely Body,53,"SFB, Ronnie Flex",NL,NULL
347,Die Heelal (feat. Early B),0,"Janie Bay, Early B",NL,NULL


In [20]:
df_central.to_csv("paises_centrales_2015_2019.csv")